# Load Packages

In [1]:
"""Credit to: https://www.kaggle.com/gpreda/jigsaw-eda"""
"""Credit to  """
# garbaga collector interface
import gc
# Miscellaneous operating system interfaces
import os
# warning control
import warnings
# use standard operator as functions
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# progress bar 
from tqdm import tqdm_notebook
# wordcloud 
from wordcloud import WordCloud, STOPWORDS
# topic modeling library
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
# natural language toolkit 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.porter import *
# library for interactive topic model visualization
import pyLDAvis
import pyLDAvis.gensim
from keras.preprocessing.text import Tokenizer

import plotly
import plotly.plotly as py
import cufflinks as cf
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='moggirain', api_key='L7BSKLkbzBKYJwc8HEp6')
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

pyLDAvis.enable_notebook()
np.random.seed(2019)
warnings.filterwarnings('ignore')


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:858: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.deco

# Load Data

In [3]:
%%time
# set the path 
PATH = "./"
# load train dataset
train = pd.read_csv(os.path.join(PATH,'train.csv'),index_col='id')
# load test dataset 
test = pd.read_csv(os.path.join(PATH,'test.csv'),index_col= 'id')
sub = pd.read_csv(os.path.join(PATH,'sample_submission.csv'),index_col = 'id')

CPU times: user 19.8 s, sys: 3.72 s, total: 23.5 s
Wall time: 24.7 s


# Exploratory Analysis

***Feature Explanation***

- The text comments are stored in `train` and `test` in `comment_text` column.  

***Target***

- The target is represented by the fraction of scores. 
- Subset of target(toxic)attributes:

`severe_toxicity`
`obscene`
`threat`
`insult`
`identity_attack`
`sexual_explicit`

***Train***

- The 5 topic categories(5): 
* **race or ethnicity**: `asian`, `black`, `jewish`, `latino`, `other_race_or_ethnicity`, `white`  
* **gender**: `female`, `male`, `transgender`, `other_gender`  
* **sexual orientation**: `bisexual`, `heterosexual`, `homosexual_gay_or_lesbian`, `other_sexual_orientation`  
* **religion**: `atheist`,`buddhist`,  `christian`, `hindu`, `muslim`, `other_religion`  
* **disability**: `intellectual_or_learning_disability`, `other_disability`, `physical_disability`, `psychiatric_or_mental_illness` <br>
<br>

- The article/comment identification information  (4):
* created_date* 
* publication_id***   
* parent_id***  
* article_id*** 
<br>

- User feedback information associated with the comments are provided  (7):
* rating  
* funny  
* wow  
* sad  
* likes  
* disagree  
* sexual_explicit  
<br>
- Annotations information (2):
* identity_annotator_count  
* toxicity_annotator_count



In [ ]:
print(train.sample(5))
print(test.head())
print(train.info())
print(train.describe())
print(test.describe())
print(f"Train and test shape:{train.shape} {test.shape}")
print(train.isnull().sum(), test.isnull().sum())


## Target Feature
- Target feature distribution

In [ ]:
plt.figure(figsize = (12,6))
plt.title('Distribution of target in the train dataset')
# kde = gaussian kernel density 
sns.distplot(train['target'],kde = True, hist = False, bins = 120, label = 'target')
plt.legend()
plt.show()

In [ ]:
train['target'].value_counts().sort_values(ascending = False)

***Explanation***
- Most target are distributed slightly around zero. 
- A second largest amount of target is around 0.2. 
- There are other peaks around 0.3, 0.4, 0.5, 0.6. (Think about how those score are generated)
- From the following confirmation, 1254764 comments are with the target exactly zero. Most comments are not annotated as toxic. 
- High toxic comments are in a small number. 

## Sub-features of Target
- Distribution of five sub-feature group 

In [ ]:
# build function to plot the features 
def plt_feature(features, title):
    plt.figure(figsize=(16,8))
    plt.title(title)
    for f in features:
        sns.distplot(train[f], kde = True, hist = False, bins = 120, label = f)
    plt.xlabel ('')
    plt.legend()
    plt.show()

In [ ]:
features = ['severe_toxicity', 'obscene','identity_attack','insult','threat']
plt_feature(features, 'Distribution of sub toxicity features in the train set')

In [ ]:
train['severe_toxicity'].value_counts().sort_values(ascending = False)
train['identity_attack'].value_counts().sort_values(ascending = False)

***Explanation***
- All sub toxicity features are centered around several peaks. 
- Insult has the highest score and identity_attack is the second. 
- The several high peak of distribution for insult and identity attach and threat are similar, both around 0.1, 0.18, 0.2, 0.3,0.4, 0.5, 0.6. 
- The distribution of severe_toxity is centered around 0.1, with several high peaks between 0 to 1. 

## Sensitive Topic Features

### Race & Ethnicity 

In [ ]:
# Race and Ethnicity 
features = ['asian','black','jewish','latino','other_race_or_ethnicity','white']
plt_feature(features,'Distribution of sensitive topic features in the train dataset ')

In [ ]:
train['white'].value_counts().sort_values(ascending = False)
train['black'].value_counts().sort_values(ascending = False)

***Explanation***
- Features are skewed toward 0.0-0.2 and 0.8-1.0. 
- Two most frequent feature is white and black and these two features had really high distribution around 1. 

### Gender

In [ ]:
features = ['female','male','transgender','other_gender']
plt_feature(features, "Distribution of gender features in the train set")

***Explanation***

- Female and male are two most frequent group of concern. 
- Similarly, the distribution is with several peaks. 
- The high values are centered around 0.0-0.2 and 0.8-1.0. 
- Female has much more counts than male distributed in the highly toxic comments. 

### Sexual Orientation

In [ ]:
features = ['bisexual', 'heterosexual', 'homosexual_gay_or_lesbian', 'other_sexual_orientation']
plt_feature(features, "Distribution of sexual orientation features in the train set")

***Explanation***

- Homosexual_gay_or_lesbian is with much more frequency, especially centered around highly toxic comments (around 0.8-1.0). 
- Other_sexual_orientation is centered around 0.1, with high peaks around 0.1. 

### Religion

In [ ]:
features = ['atheist','buddhist',  'christian', 'hindu', 'muslim', 'other_religion']
plt_feature(features, "Distribution of religion features values in the train set")

***Explanation***

- Christian and Muslim indicated higher frequency distribution. 
- Christians are distributed with relatively equal scores at each peak, but highly centered around 0.0-0.2 and 0.8-1.0, with much more counts around 1.0. 
- Muslim are with the similar trend as christian, but also highly centered around 0.0-0.2 and 0.8-1.0. 

### Disability

In [ ]:
features = ['intellectual_or_learning_disability', 'other_disability', 'physical_disability', 'psychiatric_or_mental_illness']
plt_feature(features, "Distribution of disability features values in the train set")

***Explanation***

- Psychiatric_or_mental_illness has high frequency than other disabilities, with the peaks equally distributed around certain peaks.  
- The high peaks of psychiatric_or_mental_illness are other around 0.18 and around 1.0. 
- The other_disability is around 0.1. 

## Feedback Information
- feedback values distribution. 

In [ ]:
def plot_count(feature, title, size = 1):
    f, ax = plt.subplots(1, 1, figsize = (4 * size, 4))
    total = float(len(train))
    g = sns.countplot(train[feature], order = train[feature].value_counts().index[:20],palette = 'Set1')
    g.set_title(f"count and percentage of {title}")
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2.,
               height + 3, 
               '{:1.2f}'.format(100*height/total),
                ha = 'center')
    plt.show()

### rating 

In [ ]:
# plot rating 
plot_count('rating','rating')

### funny

In [ ]:
# plot_funny
plot_count('funny','funny votes given', 4)

### wow

In [ ]:
# plot wow
plot_count('wow','wow votes given',3)

### sad

In [ ]:
plot_count('sad','sad votes given',3)

### likes

In [ ]:
plot_count('likes','likes given',3)

In [ ]:
plot_count('disagree','disagree given',3)

### sexual_explicit

In [ ]:
features = ['sexual_explicit']
plt_feature(features, 'Distribution of sexual explicit in the train set')

## Comment Text
- wordcloud of top 50 words

### WordCloud

In [ ]:
# set stopwords 
stopwords = set(STOPWORDS)
# build wordcloud
def word_cloud(data, title = None):
    wordcloud = WordCloud(
    background_color = 'white',
    stopwords = stopwords,
    max_words = 100, 
    max_font_size = 40,
    min_font_size = 6,
    scale = 5, 
    random_state = 1).generate(str(data))
    # plot the figure     
    fig = plt.figure(1, figsize = (10,10))
    plt.axis('off')
    # set title
    if title:
        fig.suptitle(title, fontsize= 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()

### comment_text 

In [ ]:
# sample 20000 comments sample and show top 100 words 
word_cloud(train['comment_text'].sample(50000), title = 'Prevalence among comments in the train data')

***explanation(from sample only)***
- top 100 words: Maybe, vote, trump, need, sure, friend, mail, globe, don't
- related to political,election,communication,affirmative words 

### insult

In [ ]:
word_cloud(train.loc[train['insult']< 0.3]['comment_text'].sample(50000),
           title = 'Prevalence insult comments in the train data')

In [ ]:
word_cloud(train.loc[train['insult']>0.7]['comment_text'].sample(20000),
           title = 'Prevalence insult comments in the train data')

***explanation***
- top words (<0.3): Muslims, Alaska, women, read, voted, wrong, like, law, news, time, said, ACA, left, leafs 
- top words (>0.7): Trump,Clinton,ideiot, mentally, stupid, clown, people, like, dumb, liar, think wrong 
- related to political,election, gender,intellectural ability, and news

### threat

In [ ]:
word_cloud(train.loc[train['threat'] < 0.3]['comment_text'],
           title = 'Prevalence threat comments in the train data')

In [ ]:
word_cloud(train.loc[train['threat']>0.7]['comment_text'],
           title = 'Prevalence threat comments in the train data')

### obscene

In [ ]:
word_cloud(train.loc[train['obscene'] <0.3]['comment_text'],
           title = 'Prevalence obscene comments in the train data')

In [ ]:
word_cloud(train.loc[train['obscene']>0.7]['comment_text'],
           title = 'Prevalence obscene comments in the train data')

### target

In [ ]:
word_cloud(train.loc[train['target']<0.3]['comment_text'],
           title = 'Prevalence of words in target in the train data')

In [ ]:
word_cloud(train.loc[train['target']>0.7]['comment_text'],
           title = 'Prevalence of words in target in the train data')

> ## New Feature Creation[](http://)

> ### Number of Characters

In [ ]:
# calculate the length 
col = "comment_text"
train['lengths'] = train[col].apply(len)
# check character distribution 
lengths = train.loc[train.lengths <1500]['lengths']
sns.distplot(lengths, color='g')
plt.show()

In [ ]:
# Check the max length of the comment text 
train.lengths.max()
train.lengths.mean()
# np.quantitle(train_df.lengths,[0.25,0.75])
train.lengths.describe()

> ***Explanation***
> [](http://)
- From the head and tail, we observe the bimodal distribution of character length in the data. Although the lengths seem to be heavily skewed to the lower lengths, we see another clear peak around the 1000 character mark.

### Number of words in the sentence

In [ ]:
# calculate the number of words
train["num_words"] = train[col].apply(lambda x: len(x.split()))
# calculate the words length < 200
words = train.loc[train.num_words < 200].num_words
sns.distplot(words, color = 'b')
plt.show()

In [ ]:
train.num_words.describe()

****From the graph***, we observe a clear unimodal left-skewed distribution of the number of words in the data.

* ### Average Word Length
- All characters / len(each word)

In [ ]:
train['avg_word_len'] = train[col].apply(lambda x: 1.0*len(''.join(x.split()))/len(x.split()))
avg_word_len = train.loc[train['avg_word_len']<8]['avg_word_len']
sns.distplot(avg_word_len, color='r')
plt.show()

In [ ]:
train.avg_word_len.describe()

### Number of Word Features 

In [ ]:
train['capitals'] = train['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
train['caps_vs_length'] = train.apply(lambda row: float(row['capitals'])/float(row['lengths']),axis=1)
train['num_exclamation_marks'] = train['comment_text'].apply(lambda comment: comment.count('!'))
train['num_question_marks'] = train['comment_text'].apply(lambda comment: comment.count('?'))
train['num_punctuation'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '.,;:'))
train['num_symbols'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in '*&$%'))
train['num_unique_words'] = train['comment_text'].apply(lambda comment: len(set(w for w in comment.split())))
train['words_vs_unique'] = train['num_unique_words'] / train['num_words']
train['num_smilies'] = train['comment_text'].apply(lambda comment: sum(comment.count(w) for w in (':-)', ':)', ';-)', ';)')))

### Correlation between new features and sub-target 

In [ ]:
features = ('lengths', 'capitals', 'caps_vs_length', 'num_exclamation_marks','num_question_marks', 'num_punctuation', 'num_words', 'num_unique_words','words_vs_unique', 'num_smilies', 'num_symbols','avg_word_len')
columns = ('target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'funny', 'wow', 'sad', 'likes', 'disagree', 'sexual_explicit','identity_annotator_count', 'toxicity_annotator_count')
rows = [{c:train[f].corr(train[c]) for c in columns} for f in features]
train_correlations = pd.DataFrame(rows, index=features)

In [ ]:
train_correlations

***Plot the Heatmap***

In [ ]:
plt.figure(figsize=(10, 6))
sns.set(font_scale=1)
ax = sns.heatmap(train_correlations, vmin=-0.1, vmax=0.1, center=0.0)

*** Explanation ***
- number of `unique_words` and `likes` are highly correlated. 
- `words_vs_unique` and `funny` are highly correlated. 
- `funny` and ` num_words` and `num_unique_words` are highly negatively correlated. **

In [ ]:
demographics = train.loc[:, ['target']+list(train)[slice(8,31)]].dropna()
weighted_toxic = demographics.iloc[:, 1:].multiply(demographics.iloc[:, 0], axis="index").sum()/demographics.iloc[:, 1:][demographics.iloc[:, 1:]>0].count()
weighted_toxic = weighted_toxic.sort_values(ascending=False)
# plot the features 
plt.figure(figsize=(30,20))
sns.set(font_scale=3)
ax = sns.barplot(x = weighted_toxic.values, y = weighted_toxic.index, alpha=0.8)
plt.ylabel('Demographics')
plt.xlabel('Weighted Toxic')
plt.show()

### Weighted toxicity across features

In [ ]:
identities = tuple(train.iloc[:, 8:31])
rows = [{c:train[f].corr(train[c]) for c in columns} for f in identities]
poptoxicity_correlations = pd.DataFrame(rows, index=identities)

In [ ]:
poptoxicity_correlations

In [ ]:
plt.figure(figsize=(12, 8))
sns.set(font_scale=1)
ax = sns.heatmap(poptoxicity_correlations, vmin=-0.1, vmax=0.1, center=0.0)

*** Explanation ***
- `identity_attach` is highly correlated with black, homesexual, juewish, muslim. 
- `sexual_explicit` is highly correlated with female, homexual, male. 
- `target` is highly correlated with black, homesexual, muslim and white. 

## Time Series 

In [98]:
type(train.created_date)

pandas.core.series.Series

In [99]:
# create a new dataframe with date 
withdate = train.loc[:, ['created_date', 'target']+list(train)[slice(7,31)]].dropna()
# calculate the race weighted ratio
raceweighted = withdate.iloc[:, 2:]/withdate.iloc[:, 2:].sum()
# calculate the ratio multiplied with target 
race_target_weighted = raceweighted.multiply(withdate.iloc[:, 1], axis="index")
# created a new column to change the datatype 
race_target_weighted['created_date'] = pd.to_datetime(withdate['created_date']).values.astype('datetime64[M]')
# order by time stamp and sum the values and sort by months
weighted_demo = race_target_weighted.groupby(['created_date']).sum().sort_index()

In [ ]:
withdate.head()

### Race & Ethnicity

In [ ]:
weighted_demo[['white', 'asian', 'black', 'jewish', 'latino', 'other_race_or_ethnicity']].iplot(title = 'Time Series Toxicity & Race', filename='Time Series Toxicity & Race' )

### Religion

In [ ]:
weighted_demo[['atheist', 'buddhist', 'christian', 'hindu', 'muslim', 'other_religion']].iplot(title = 'Time Series Toxicity & Religion', filename='Time Series Toxicity & Religion')

### Sexual Orientation

In [ ]:
weighted_demo[['heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']].iplot(title = 'Time Series Toxicity & Sexual Orientation', filename='Time Series Toxicity & Sexual Orientation')


### Gender

In [ ]:
weighted_demo[['male', 'female', 'transgender', 'other_gender']].iplot(title = 'Time Series Toxicity & Gender', filename='Time Series Toxicity & Gender')

### Disability

In [ ]:
weighted_demo[['physical_disability', 'intellectual_or_learning_disability', 'psychiatric_or_mental_illness', 'other_disability']].iplot(title = 'Time Series Toxicity & Disability', filename='Time Series Toxicity & Disability')

## Jan 2017

In [100]:
alldate_toxicity = train[train['target'] >= 0.5].loc[:, ['created_date', 'target', 'comment_text']].dropna()
alldate_toxicity['created_date'] = pd.to_datetime(alldate_toxicity['created_date']).values.astype('datetime64[M]')
jan_2017_toxicity = alldate_toxicity[alldate_toxicity['created_date'] == '2017-01-01']

def check_frequency(data = alldate_toxicity['comment_text'], n = 20):
    stop = stopwords.words('english')
    data  = data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    data = data.str.replace('[^\w\s]','')
    data = data.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    freq = pd.Series(' '.join(data).split()).value_counts()[:n]
    return freq

top_10_toxicity_othertime = check_frequency(data = alldate_toxicity[alldate_toxicity['created_date'] != '2017-01-01']['comment_text'], n = 10)
top_10_toxicity_jan_2017 = check_frequency(data = jan_2017_toxicity['comment_text'], n = 10)

In [101]:
percent_toxicity_othertime = top_10_toxicity_othertime/alldate_toxicity[alldate_toxicity['created_date'] != '2017-01-01']['comment_text'].str.split().str.len().sum()
percent_toxicity_jan_2017 = top_10_toxicity_jan_2017/jan_2017_toxicity['comment_text'].str.split().str.len().sum()
top_toxicity = pd.concat([percent_toxicity_jan_2017, percent_toxicity_othertime], axis=1, sort=False)
top_toxicity.columns = ['Jan_2017', 'Other_Time']
top_toxicity['Difference'] = top_toxicity['Jan_2017'] - top_toxicity['Other_Time']

In [102]:
top_toxicity.head(30)

,Jan_2017,Other_Time,Difference
trump,0.005175,0.004231,0.000943
people,0.003828,0.004018,-0.000191
like,0.003643,0.003650,-0.000007
dont,0.002613,0.002662,-0.000049
one,0.002587,0.002684,-0.000097
would,0.002556,0.002828,-0.000271
us,0.002512,0.001903,0.000609
get,0.002296,0.002427,-0.000131
stupid,0.001985,0.002072,-0.000087
think,0.001680,0.001696,-0.000017


In [104]:
trace1 = go.Bar(
    x=top_toxicity.index,
    y=top_toxicity['Jan_2017'],
    name='Jan_2017'
)
trace2 = go.Bar(
    x=top_toxicity.index,
    y=top_toxicity['Other_Time'],
    name='Other_Time'
)

data = [trace2, trace1]
layout = go.Layout(
    barmode='group'
)
layout = go.Layout(yaxis=dict(tickformat=".2%"))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, title = 'Top Toxicity Comarision', filename='top_toxicity_comarision')

### Which Time are People More Toxic?

In [105]:
train['datetime64'] = pd.to_datetime(train['created_date']).values.astype('datetime64[h]')
train['hour'] = train['datetime64'].dt.hour
all_comments_by_hour = train['target'].groupby(train['hour']).sum().sort_index()/train['target'].groupby(train['hour']).sum().sum()
toxic_comments_by_hour = train[train['target'] >= 0.5]['target'].groupby(train['hour']).sum().sort_index()/train[train['target'] >= 0.5]['target'].groupby(train['hour']).sum().sum()
comments_hour_check = pd.concat([all_comments_by_hour, toxic_comments_by_hour], axis=1, sort=False)
comments_hour_check.columns = ['all_comments', 'toxic_comments']

In [106]:
labels = ['Midnight', 'Morning', 'Noon', 'Evening', 'Midnight']
tickvals = ['0', '6', '12', '18', comments_hour_check.index.max()]

trace1 = go.Scatter(
    x=comments_hour_check.index,
    y=comments_hour_check['all_comments'],
    name = 'comment percent per H',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        width = 1)
)
trace2 = go.Scatter(
    x=comments_hour_check.index,
    y=comments_hour_check['toxic_comments'],
    name = 'toxic comment percent per H',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 1,)
)

trace3 = go.Bar(
    x=comments_hour_check.index,
    y=comments_hour_check['toxic_comments']-comments_hour_check['all_comments'],
    name = 'More Toxic Comment Ratio'
)

data = [trace1, trace2, trace3]

layout = go.Layout(yaxis=dict(tickformat=".2%"),
                   title = 'Which Time are People More Toxic',
                   xaxis=go.layout.XAxis(
                       ticktext=labels, 
                       tickvals=tickvals
                   ),
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='Which Time are People More Toxic')

## Topic Modeling 

Here we observe a simple bell-shaped normal distribution of the average word length with a mean of around 4.9

### tokenization

In [ ]:
# preprocess the trian data and model a subset of the comments from train data
def preprocess (text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(token)
    return result

In [ ]:
# test for one comment 
# comment_sample = train['comment_text'][:1].values[0]
# print(f'Original comment: {comment_sample}')
# print(f'Tokenized comment: {preprocess(comment_sample)}')

In [ ]:
# tokenize the sampling data 
preprocessed = train['comment_text'].sample(200000).map(preprocess)

In [ ]:
# check the tokenization
preprocessed.sample(3)

### Create dictionary and filter data

In [ ]:
# create a dictionary
dictionary = gensim.corpora.Dictionary(preprocessed)
# filter extreme data 
dictionary.filter_extremes(no_below = 10, no_above = 0.5, keep_n = 75000)

### Bag of Words

In [ ]:
# measure the presence of words 
# output word idx and its frequency 
bow = [dictionary.doc2bow(doc) for doc in preprocessed]
# tfidf
tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]

### LDA

In [ ]:
# build LDA model 
lda = gensim.models.LdaMulticore(corpus_tfidf, num_topics= 20,
                                id2word = dictionary, passes = 2, workers = 2)

In [ ]:
# generate the topics 
topics = lda.print_topics(num_words = 8)
for i, topic in enumerate(topics[:10]):
    print(f"Train topic{i} {topic}")

In [ ]:
# predict the type fo topic for one document with the index = 10
bd10 = bow[10]
for i in range(len(bd10)):
    print(f"Word {bd10[i][0]} (\'{dictionary[bd10[i][0]]}\')  appears  {bd10[i][1]} time. ")

In [ ]:
for idx, score in sorted(lda[bd10], key = lambda tup: -1*tup[1]):
    print(f"\nScore: {score}\t \nTopic:{lda.print_topic(idx, 8)}")

### pyLDAvis

In [ ]:
vis = pyLDAvis.gensim.prepare(lda,bow,dictionary)

In [ ]:
pyLDAvis.save_html(vis, 'LDAis_train.html')

### Create Topic Sentence

In [ ]:
# check the most important topics per each comment 
def topic_sentences(ldamodel = lda, corpus = bow, texts = preprocessed):
    
    # initalization
    topic_df = pd.DataFrame()
    
    # get main topic in each comment 
    # lda[corpus] ouput tuple (topic_num, proportion)
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key = lambda x: (x[1]), reverse = True)
         # get the domain topic, %contribution and keywords for each document
            # sorted by proportion
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # = dominant topic
                # display the topic as (words, proportion)
                wp = ldamodel.show_topic(topic_num)
                # join the keywords in a list
                topic_keywords = ", ".join([word for word, prop in wp])
                # build a dataframe
                topic_df = topic_df.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_keywords]), ignore_index = True)
            else:
                break
    # concatnet topic and text 
    text = pd.Series(texts)
    topic_df = pd.concat([topic_df, text],axis = 1)
    return(topic_df)

In [ ]:
topic_keywords = topic_sentences(ldamodel = lda, corpus = bow, texts = preprocessed)

In [ ]:
topic_keywords.to_csv("lda_model_result.csv")

In [ ]:
dominant_topic = topic_keywords.reset_index()
dominant_topic.columns = ['Comment','Dominant_topic','Topic_percent_contribution','Keywords','Text']
dominant_topic.head()

### Topic Modeling for test data

In [ ]:
# tokenize comments 
preprocessed_comments = test['comment_text'].map(preprocess)
# create dictionary 
dictionary = gensim.corpora.Dictionary(preprocessed_comments)
# filter the dictionary 
dictionary.filter_extremes(no_below = 10, no_above = 0.5, keep_n = 75000)
# bag of words
bow_test = [dictionary.doc2bow(doc) for doc in preprocessed_comments]
# tfidf ouput the tfidf scores tf*idf
tfidf = models.TfidfModel(bow_test)
# dictionary for each bag tfidf
corpus_tfidf = tfidf[bow_test]

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics= 20, id2word= dictionary, passes = 2, workers= 2)

In [ ]:
topics = lda_model.print_topics(num_words = 5)
for i, topic in enumerate(topics[:10]):
    
    print(f"Test topic {i}: {topic}")

In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, bow_test, dictionary)
pyLDAvis.save_html(vis,"LDAVis_test.html")

### Sentiment (negativity)
#### The blue distribution is label 0 (non-toxic) and the red[](http://) distribution is label 1 (toxic)

In [ ]:
# >0.5 is toxic and <0.5 is non-toxic
SIA = SentimentIntensityAnalyzer()
polarity_0 = train.loc[train.target < 0.5][col].apply(lambda x:SIA.polarity_scores(x))
polarity_1 = train.loc[train.target > 0.5][col].apply(lambda x:SIA.polarity_scores(x))

In [ ]:
# plot the sentiment
sns.distplot([polarity['neg'] for polarity in polarity_0], color = 'blue')
sns.distplot([polarity['neg'] for polarity in polarity_1], color='red')
plt.show()

- Clearly, the blue distribution has a higher mean than the red distribution. Both distributions shared sismilar std. Although both distributions are skewed leftwards, the red distribution has a stronger leftward skew.
- The trend indicated that the toxic comments generally tend to be more negative on average. This is probably because most comments considered "toxic" generally spread negative emotions like hate, anger or insult against certain people, beliefs or opinions.

### Sentiment (positivity)

In [ ]:
sns.distplot([polarity['pos'] for polarity in polarity_0], color='blue')
sns.distplot([polarity['pos'] for polarity in polarity_1], color='red')
plt.show()

Both two distributions shared similar characteristics : bimodality, shape, skew, mean etc.It indicated there is no significant difference between positivity in toxic and non-toxic comments.

### Sentiment (neutrality)

In [ ]:
sns.distplot([polarity['neu'] for polarity in polarity_0], color='blue')
sns.distplot([polarity['neu'] for polarity in polarity_1], color='red')
plt.show()

### Sentiment (compoundness / complexity of comment)

In [ ]:
sns.distplot([polarity['compound'] for polarity in polarity_0], color='darkorange')
sns.distplot([polarity['compound'] for polarity in polarity_1], color='purple')
plt.show()

***Explanation***
- Both distributions have a somewhat similar shape : evenly spread out with little skew in one direction and a small insignificant peak in the center (at around 0). The skewness is different. 
- This shows that toxic comments are generally less gramatically complex than non-toxic comments. It is probably because toxic comments are generally more blunt and short. 
- On the other hand, non-toxic comments generally try to share a perspective or make a point, and thus, they tend to be more gramatically complex or compounded on average.

## Modeling Preparation

**Rules for model preparation using Deeplearning**

- Don't use standard preprocessing steps like stemming or stopword removal when you have pre-trained embeddings
- Get your vocabulary as close to the embeddings as possible

### Building Vocabulary

`EMBED_SIZE` - embedding size - the size of word vector - should match the embedding source (GloVe);
<br>
`MAX_FEATURES` - Maximum number of features - the number of unique words to use or number of rows in the embedding vector;
<br>
`MAXLEN` - The maximum length of comments text

In [ ]:
 # size of word vector; this should be set to 300 to match the embedding source
EMBED_SIZE = 300
# how many unique words to use (i.e num rows in embedding vector)
MAX_FEATURES = 100000 
# max length of comments text
MAXLEN = 220 

### Vocabulary Building 

In [ ]:
def build_vocabulary(texts):
    """
    credits to: https://www.kaggle.com/gpreda/jigsaw-eda#Prepare-the-model
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: list of list of words
    output: dictionary of words and their count
    """
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in tqdm_notebook(sentences):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
# concatnate train and test and build an unique vocabulary with both datasets
# populate the vocabulary 
df = pd.concat([train,test], sort = False)
vocabulary = build_vocabulary(df['comment_text'])
print({k: vocabulary[k] for k in list(vocabulary)[:10]})

### Index and Matrix Embedding

In [ ]:
def load_embeddings(file):
    """
    credits to: https://www.kaggle.com/takuok/glove840b300dtxt/downloads/glove.840B.300d.txt/1
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: embeddings file
    output: embedding index
    """
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
    return embeddings_index
        

In [ ]:
%%time
# convert it to glove path
GLOVE_PATH = './'
print("Extracting GloVe embedding started")
embed_glove = load_embeddings(os.path.join(GLOVE_PATH,'glove.840B.300d.txt'))
print("Embedding completed")

In [ ]:
len(embed_glove)

In [ ]:
def embedding_matrix(word_index, embeddings_index):
    '''
    credits to:https://www.kaggle.com/takuok/glove840b300dtxt/downloads/glove.840B.300d.txt/1
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: word index, embedding index
    output: embedding matrix
    '''
    # enbedding matrix-stack all the index 
    all_embs = np.stack(embeddings_index.values())
    # calculate embedding mean and std
    emb_mean, emb_std = all_embs.mean(), all_embs.std()
    # set teh size 
    EMBED_SIZE = all_embs.shape[1]
    # number of words
    nb_words = min(MAX_FEATURES, len(word_index))
    # initiate embedding matrix 
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBED_SIZE))
    # iterate to get word embedding for each word
    for word, i in tqdm_notebook(word_index.items()):
        if i >= MAX_FEATURES:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

### Check Coverage
- Check coverage of  embeddings for the vocabulary created from the train and test dataset 

In [ ]:
def check_coverage(vocab, embeddings_index):
    """
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: vocabulary, embedding index
    output: list of unknown words; also prints the vocabulary coverage of embeddings and the % of comments text covered by the embeddings
    
    """
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in tqdm_notebook(vocab.keys()):
        try:
            # build dictionary for known_words
            known_words[word] = enbeddings_index[word]
            # nb_known_words 
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab [word]
            pass
    print('Found embeddings for {:.3%} of vocabulary'.format(len(known_words)/len(vocab)))
    print('Found embeddings for {:.3%} of all text'.format(nb_known_words/(nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]
    return unknown_words

In [ ]:
print("Verify the intial vocabulary coverage")
oov_glove = check_coverage(vocabulary, embed_glove)

In [ ]:
oov_glove[:20]

### Transform to lower case 

In [ ]:
def add_lower(embedding, vocab):
    '''
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: vocabulary, embedding matrix
    output: modify the embeddings to include the lower case from vocabulary
    '''
    count = 0
    for word in tqdm_notebook(vocab):
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x: x.lower())
test['comment_text'] = test['comment_text'].apply(lambda x: x.lower())

In [ ]:
print("Check coverage for vocabulary with lower case")
# check oov_glove coverage 
oov_glove = check_coverage(vocabulary, embed_glove)
add_lower(embed_glove, vocabulary) # operates on the same vocabulary
oov_glove = check_coverage(vocabulary, embed_glove)

In [ ]:
oov_glove[:10]

### Remove Contradictions 

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}
len(contraction_mapping)

In [ ]:
def known_contractions(embed):
    '''
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: embedding matrix
    output: known contractions (from embeddings)
    '''
    known = []
    for contract in tqdm_notebook(contraction_mapping):
        if contract in embed:
            known.append(contract)
    return known

In [ ]:
print("Known contractions in GloVe embeddings:")
print(known_contractions(embed_glove))

In [ ]:
def clean_contractions(text, mapping):
    '''
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: current text, contraction mappings
    output: modify the comments to use the base form from contraction mapping
    '''
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test['comment_text'] = test['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

In [ ]:
df = pd.concat([train ,test], sort=False)
vocab = build_vocabulary(df['comment_text'])
print("Check embeddings after applying contraction mapping")
oov_glove = check_coverage(vocab, embed_glove)

In [ ]:
oov_glove[:10]

### Remove Punctuation

In [ ]:
punct_mapping = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping += '©^®` <→°€™› ♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√'

def unknown_punct(embed, punct):
    '''
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: current text, contraction mappings
    output: unknown punctuation
    '''
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

In [ ]:
print("Find unknown punctuation:")
print(unknown_punct(embed_glove, punct_mapping))

In [ ]:
puncts = {"‘": "'", "´": "'", "°": "", "€": "e", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '…': ' '}

def clean_special_chars(text, punct, mapping):
    '''
    credits to: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings 
    credits to: https://www.kaggle.com/anebzt/quora-preprocessing-model
    input: current text, punctuations, punctuation mapping
    output: cleaned text
    '''
    for p in mapping:
        text = text.replace(p, mapping[p])
    for p in punct:
        text = text.replace(p, f' {p} ') 
    return text

In [ ]:
train['comment_text'] = train['comment_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))
test['comment_text'] = test['comment_text'].apply(lambda x: clean_special_chars(x, punct_mapping, puncts))

In [ ]:
%%time
df = pd.concat([train ,test], sort=False)
vocab = build_vocabulary(df['comment_text'])
print("Check coverage after punctuation replacement")
oov_glove = check_coverage(vocab, embed_glove)

### Tokenize 

In [ ]:
%%time
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(list(train))
train = tokenizer.texts_to_sequences(train)
test = tokenizer.texts_to_sequences(test)